In [1]:
from openai import OpenAI
import json
import os
import datetime
import requests

In [2]:
os.environ["OPENAI_API_KEY"]  = ""
client = OpenAI()

In [3]:
weather_api_key = ""
base_url_weather="https://api.openweathermap.org/data/2.5/weather?"
base_url_coordinates = "https://api.openweathermap.org/geo/1.0/direct?"

In [4]:
def get_current_weather(location, unit="fahrenheit"):
    #print("called")
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [5]:
def get_system_time():
    now = datetime.datetime.now()

    # Access different components of the datetime object
    year = now.year
    month = now.month
    day = now.day
    hour = now.hour
    minute = now.minute
    second = now.second
    microsecond = now.microsecond
    
    # Format the datetime object in a specific way
    formatted_time = now.strftime("%Y-%m-%d %H:%M:%S")
    return formatted_time

In [6]:
def get_real_weather(city_name, unit="fahrenheit"):
    try:
        real_unit=""
        if unit=="fahrenheit":
            real_unit="Imperial"
        elif unit=="celsius":
            real_unit="Metric"
        elif unit=="kelvin":
            real_unit="Default"
        final_coord_url=base_url_coordinates+ "appid=" + weather_api_key + "&q=" + city_name +"&limit=1"
        coord_response = requests.get(final_coord_url).json()
        lat=coord_response[0]["lat"]
        lon=coord_response[0]["lon"]
        final_weather_url=base_url_weather+ "appid=" + weather_api_key +"&lat="+str(lat)+"&lon="+str(lon)+"&units="+real_unit
        #print(final_weather_url)
        final_response = requests.get(final_weather_url).json()
        print(final_response)
        return "Min temp is "+str(final_response["main"]["temp_min"])+ "  Max temp is "+str(final_response["main"]["temp_max"])
    except Exception as e:
        print(e)
        return "could not find weather data"
    

In [7]:
def get_real_weather2(lat,lon, unit="fahrenheit"):
    try:
        print("called")
        real_unit=""
        if unit=="fahrenheit":
            real_unit="Imperial"
        elif unit=="celsius":
            real_unit="Metric"
        elif unit=="kelvin":
            real_unit="Default"
        #final_coord_url=base_url_coordinates+ "appid=" + weather_api_key + "&q=" + city_name +"&limit=1"
        #coord_response = requests.get(final_coord_url).json()
        #lat=coord_response[0]["lat"]
        #lon=coord_response[0]["lon"]
        final_weather_url=base_url_weather+ "appid=" + weather_api_key +"&lat="+str(lat)+"&lon="+str(lon)+"&units="+real_unit
        #print(final_weather_url)
        final_response = requests.get(final_weather_url).json()
        print(final_response)
        return "Min temp is "+str(final_response["main"]["temp_min"])+ "  Max temp is "+str(final_response["main"]["temp_max"])
    except Exception as e:
        print(e)
        return "could not find weather data"

In [8]:
def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "What's the weather like in dubai, mussoorie,pune and goa in celcius? and what is the current time"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_real_weather2",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "lat": {
                            "type": "string",
                            "description": "it must be a latitude of the city, eg 28.6517178",
                        },
                        "lon": {
                            "type": "string",
                            "description": "it must be a longitude city, eg delhi,mumbai,tokyo,etc",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit","kelvin"]},
                    },
                    "required": ["location"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_system_time",
                "description": "Get the current time",
                "parameters": {
                    "type": "object",
                    "properties": {},
                   
                    "required": [],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_real_weather2": get_real_weather2,
            "get_system_time":get_system_time
                }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            print(function_name)
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            #print("functions might be called here 3")
            print(function_args)
            if(function_name=="get_real_weather"):
                function_response = function_to_call(
                    city_name=function_args.get("city_name"),
                    unit=function_args.get("unit"),
                )
            if(function_name=="get_real_weather2"):
                function_response = function_to_call(
                lat=function_args.get("lat"),
                lon=function_args.get("lon"),
                unit=function_args.get("unit"),
                )
            elif(function_name=="get_system_time"):
                function_response=function_to_call()
                #print(function_response)
            else:
                function_response="sorry cannot perform this action"
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            messages=messages,
        )  
        # get a new response from the model where it can see the function response
        #print(messages)
        return second_response

In [9]:
print(run_conversation())

get_real_weather2
{'lat': '25.276987', 'lon': '55.296249', 'unit': 'celsius'}
called
{'coord': {'lon': 55.2962, 'lat': 25.277}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}], 'base': 'stations', 'main': {'temp': 19.81, 'feels_like': 19.2, 'temp_min': 18.21, 'temp_max': 20.01, 'pressure': 1014, 'humidity': 52}, 'visibility': 10000, 'wind': {'speed': 6.69, 'deg': 260}, 'clouds': {'all': 20}, 'dt': 1709239234, 'sys': {'type': 1, 'id': 7537, 'country': 'AE', 'sunrise': 1709260908, 'sunset': 1709302848}, 'timezone': 14400, 'id': 292223, 'name': 'Dubai', 'cod': 200}
get_real_weather2
{'lat': '30.4591', 'lon': '78.0664', 'unit': 'celsius'}
called
{'coord': {'lon': 78.0664, 'lat': 30.4591}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 6.56, 'feels_like': 4.6, 'temp_min': 6.56, 'temp_max': 6.56, 'pressure': 1016, 'humidity': 51, 'sea_level': 1016, 'grnd_level': 807}, 'visi